In [2]:
#워드 임베딩
#자연어를 컴퓨터가 이해하도록 변환해야한다. 
#단어를 벡터로 표현하는 방법으로 단어를 밀집표현으로 변환하다.
#희소표현: 원핫벡터 대부분 0으로 하나만 1로
#차원이 커지는 문제
#밀집표현: 희소표현과 반대개념
#밀집벡터형대로 표현하는 방법을 워드임베딩
#밀집벡터를 워드임베딩과정을 통해 나온 결과 : 임베딩 벡터

#워드투벡터: 원핫벡터는 단어간 유사도를 계산할 수 없다.
#분산표현은 저차원에 단어의 의미를 여러차원에다가 분산해서 표현한다.
#이런 표현방법으로 단어 간 유사도를 계산한다.

#CBOW(연속적인 BoW)
#Word2Vec에서 사용하는 방법 : 주변에 있는 단어들을 가지고 중간에 있는 단어들을 예측하는 방법
#Skip-Gram은 중간에 있는 단어로 주변단어들을 예측하는 방법
#The fat cat sat on the mat : sat이 중간단어 나머지마 주변단어

#한국어 Word2Vec 만들기(네이버 영화 리뷰)
import pandas as pd
import matplotlib.pyplot as plt
import urllib.request
import re
from gensim.models.word2vec import Word2Vec
from konlpy.tag import Okt
urllib.request.urlretrieve("https://raw.githubusercontent.com/e9t/nsmc/master/ratings.txt", filename="ratings.txt")
train_data=pd.read_table('ratings.txt')

train_data[:5]#상위 5개 출력


,id,document,label
0,8112052,어릴때보고 지금다시봐도 재밌어요ㅋㅋ,1
1,8132799,"디자인을 배우는 학생으로, 외국디자이너와 그들이 일군 전통을 통해 발전해가는 문화산...",1
2,4655635,폴리스스토리 시리즈는 1부터 뉴까지 버릴께 하나도 없음.. 최고.,1
3,9251303,와.. 연기가 진짜 개쩔구나.. 지루할거라고 생각했는데 몰입해서 봤다.. 그래 이런...,1
4,10067386,안개 자욱한 밤하늘에 떠 있는 초승달 같은 영화.,1


In [5]:
print(len(train_data))

200000


In [6]:
print(train_data.isnull().values.any())#NULL값 존재 여부

True


In [7]:
train_data=train_data.dropna(how='any')#NULL값 존재하는 행 제거
print(train_data.isnull().values.any())

False


In [8]:
print(len(train_data))

199992


In [11]:
train_data['document']=train_data['document'].str.replace("[^ㄱ-ㅎㅏ-ㅣ가-힣 ]"," ")
#정규 표현식으로 한글 외 문자제거
train_data[:5]

,id,document,label
0,8112052,어릴때보고지금다시봐도재밌어요ㅋㅋ,1
1,8132799,디자인을배우는학생으로외국디자이너와그들이일군전통을통해발전해가는문화산업이부러웠는데사실우...,1
2,4655635,폴리스스토리시리즈는부터뉴까지버릴께하나도없음최고,1
3,9251303,와연기가진짜개쩔구나지루할거라고생각했는데몰입해서봤다그래이런게진짜영화지,1
4,10067386,안개자욱한밤하늘에떠있는초승달같은영화,1


In [ ]:
okt=Okt()
tokenized_data=[]
stopwords = ['의','가','이','은','들','는','좀','잘','걍','과','도','를','으로','자','에','와','한','하다']
for sentence in train_data['document']:
    temp_X=okt.morphs(sentence,stem=True)#토근화
    temp_X=[word for word in temp_X if not word in stopwords]#불용어 제거
    tokenized_data.append(temp_X)
print('리뷰의 최대 길이: ',max(len(l) for l in tokenized_data))
print('리뷰의 평균 길이: ',sum(map(len,tokenized_data))/len(tokenized_data))
plt.hist([len(s) for s in tokenized_data],bins=50)
plt.xlabel('length of samples')
plt.ylabel('number of samples')
plt.show()

In [15]:
from gensim.models import Word2Vec
model=Word2Vec(sentences=tokenized_data,size=100,window=5,min_count=5,workers=4,sg=0)
#sg는 CBOW, min_count는 빈도수 낮은거 제거
model.wv.vectors.shape
print(model.wv.most_similar("최민식"))

[('잇다', 0.9978945255279541), ('걸', 0.997874915599823), ('어', 0.997853696346283), ('적', 0.9978455901145935), ('함께', 0.9978387951850891), ('따다', 0.9978170990943909), ('쓰다', 0.9978167414665222), ('조', 0.9978134036064148), ('니', 0.9978111386299133), ('정말', 0.9978067874908447)]
